### Find accuracy of using xgboost on the loan data

In [1]:
import numpy as np
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import joblib


### Load the data

In [2]:
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

### Split the training data into training and validation data

In [3]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size=0.1,
    stratify=y_train,
    random_state=42
)

### Balance the training data using SMOTE

In [4]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

### Build the xgboost model

In [5]:
xgb_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='aucpr',
    random_state=42
)

In [6]:
xgb_model.fit(
    X_train_resampled, y_train_resampled,
    eval_set=[(X_val, y_val)],
    verbose=True
)

[0]	validation_0-aucpr:0.98996
[1]	validation_0-aucpr:0.99295
[2]	validation_0-aucpr:0.99708
[3]	validation_0-aucpr:0.99716
[4]	validation_0-aucpr:0.99746
[5]	validation_0-aucpr:0.99785
[6]	validation_0-aucpr:0.99749
[7]	validation_0-aucpr:0.99751
[8]	validation_0-aucpr:0.99759
[9]	validation_0-aucpr:0.99750
[10]	validation_0-aucpr:0.99771
[11]	validation_0-aucpr:0.99755
[12]	validation_0-aucpr:0.99757
[13]	validation_0-aucpr:0.99762
[14]	validation_0-aucpr:0.99762
[15]	validation_0-aucpr:0.99759
[16]	validation_0-aucpr:0.99752
[17]	validation_0-aucpr:0.99750
[18]	validation_0-aucpr:0.99749
[19]	validation_0-aucpr:0.99743
[20]	validation_0-aucpr:0.99742
[21]	validation_0-aucpr:0.99736
[22]	validation_0-aucpr:0.99751
[23]	validation_0-aucpr:0.99756
[24]	validation_0-aucpr:0.99758
[25]	validation_0-aucpr:0.99762
[26]	validation_0-aucpr:0.99758
[27]	validation_0-aucpr:0.99758
[28]	validation_0-aucpr:0.99753
[29]	validation_0-aucpr:0.99755
[30]	validation_0-aucpr:0.99768
[31]	validation_0-

/opt/anaconda3/envs/tf-mac/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [19:31:02] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[94]	validation_0-aucpr:0.99753
[95]	validation_0-aucpr:0.99751
[96]	validation_0-aucpr:0.99749
[97]	validation_0-aucpr:0.99760
[98]	validation_0-aucpr:0.99755
[99]	validation_0-aucpr:0.99757


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

### Evaluate the model accuracy using test data

In [7]:

y_pred = xgb_model.predict(X_test)


              precision    recall  f1-score   support

           0      0.981     0.975     0.978       323
           1      0.985     0.989     0.987       531

    accuracy                          0.984       854
   macro avg      0.983     0.982     0.983       854
weighted avg      0.984     0.984     0.984       854



In [8]:
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()

print(classification_report(y_test, y_pred, digits=3))

In [9]:
print("Model accuracy:", xgb_model.score(X_test, y_test))

Model accuracy: 0.9836065573770492


### Comparing Logistics Regression to XGboost, XGboost has the higher accuracy

In [10]:
joblib.dump(xgb_model, 'loan_xgb_model.pkl') # Save the model for future use

['loan_xgb_model.pkl']